In [8]:
from glob import glob
import docx2txt
import numpy as np
import pandas as pd
import cv2
import os
from tqdm import tqdm
from glob import glob
import spacy
import re
import string
import json

2022-04-19 14:08:38.666307: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/codiant/anaconda3/lib/python3.9/site-packages/cv2/../../lib64:
2022-04-19 14:08:38.666381: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [6]:
#import pywin32com.client as win32
#from win32com.client import constants

ERROR: Could not find a version that satisfies the requirement pywin32 (from versions: none)
ERROR: No matching distribution found for pywin32


In [6]:
import os
import re

# Create list of paths to .doc files
paths = glob('D:\\All_resumes\\*.doc', recursive=True)

def save_as_docx(path):
    # Opening MS Word
    word = win32.gencache.EnsureDispatch('Word.Application')
    doc = word.Documents.Open(path)
    doc.Activate ()

    # Rename path with .docx
    new_file_abs = os.path.abspath(path)
    new_file_abs = re.sub(r'\.\w+$', '.docx', new_file_abs)

    # Save and Close
    word.ActiveDocument.SaveAs(
        new_file_abs, FileFormat=constants.wdFormatXMLDocument
    )
    doc.Close(False)

for path in paths:
    save_as_docx(path)

com_error: (-2147418111, 'Call was rejected by callee.', None, None)

In [ ]:

dic = []

resPaths= glob('D:/All_resumes/*.docx')

for resPath in tqdm(resPaths,desc = 'Resumes-doc'):
    #print(resPath)
    directory, filename = os.path.split(resPath)
    
    my_text = docx2txt.process(resPath)
    
    my_text = my_text.replace(",",", ")
    my_text = re.sub(r'(\w+?)\(',r'\1 (',my_text)
    my_text = re.sub(r'([Ee][Mm][Aa][Ii][Ll]:)(\w)',r'\1 \2',my_text)
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    my_text = url_pattern.sub(r'', my_text)
    
    punctuation = "!#$%'*;<=>?^`|~"
    tablePunctuation = str.maketrans('','',punctuation)
    removepunctuation = my_text.translate(tablePunctuation)
    removepunctuation = re.sub(r' +',' ',removepunctuation)
    
    dictionary ={"text":removepunctuation ,"meta":{"source":resPath}}
    dic.append(dictionary)

In [ ]:
res_pdf_paths = glob('D:/All_resumes/*.pdf')
page = 0
for res_pdf_path in tqdm(res_pdf_paths,desc = 'Resumes-PDF'):
    #print(res_pdf_path)
    directory, filename = os.path.split(res_pdf_path)
    
    pdfFileObj = open(res_pdf_path, 'rb') 
    
    # creating a pdf reader object 
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj) 

    # printing number of pages in pdf file 
    page = pdfReader.numPages
    my_pdf_text = ''
    for i in range(0,page):
        pageObj = pdfReader.getPage(i) 
        x = ''
        x = pageObj.extractText()
        my_pdf_text = my_pdf_text+x
    
    
    
    my_pdf_text = my_pdf_text.replace(",",", ")
    my_pdf_text = re.sub(r'(\w+?)\(',r'\1 (',my_pdf_text)
    my_pdf_text = re.sub(r'([Ee][Mm][Aa][Ii][Ll]:)(\w)',r'\1 \2',my_pdf_text)
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    my_pdf_text = url_pattern.sub(r'', my_pdf_text)
    
    punctuation = "!#$%'*;<=>?^`|~"
    tablePunctuation = str.maketrans('','',punctuation)
    removepunctuation = my_pdf_text.translate(tablePunctuation)
    removepunctuation = re.sub(r' +',' ',removepunctuation)
    
    dictionary ={"text":removepunctuation ,"meta":{"source":res_pdf_path}}
    dic.append(dictionary)
    

In [ ]:
with open("all_resumes.json", "w") as final:
    json.dump(dic, final)
    print('\n')

In [ ]:
def get_pdf_text(file):
        '''
        file : PDF file object
        return text : str
        '''
        try:
            with pdfplumber.open(file) as pdf:
                pages = pdf.pages
                combineText = []
            for page in range(0,len(pages)):
                page_obj = pdf.pages[page]
                text = page_obj.extract_text()
                combineText.append(text)
            text = " ".join(combineText)
            return text
        except Exception as e:
            print("Unable to read the file",e)
            return False
    def get_docx_text(file):
        '''
        file : DOCX file object
        return text : str
        '''
        try:
            text = docx2txt.process(file)
            return text
        except Exception as e:
            print("Unprocessed entity", e)
            return False